In [211]:
#import
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
#You will have to download the set of stop words the first time
#nltk.download('stopwords')
#nltk.download('punkt')
from collections import Counter

In [212]:
#define
top_N = 10000
stopwords = nltk.corpus.stopwords.words('english')
#we need to rename this directory to wherever the CSV files are located
yelp_attr = pd.read_csv('yelp_business_attributes.csv')
yelp_rev = pd.read_csv('yelp_review.csv',nrows = 10000)

In [213]:
#Combine Ambience and Reviews into a Dataframe
yelp_reviewd = pd.merge(yelp_rev, yelp_attr, on='business_id')
i = 0
for index, row in yelp_reviewd.iterrows():    
    yelp_reviewd.loc[index, "is_bougie"] = row["stars"] == 5
yelp_reviewd.head(10)
#list(yelp_reviewd.columns.values)



,review_id,user_id,business_id,stars,date,text,useful,funny,cool,AcceptsInsurance,...,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService,is_bougie
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
1,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5,2016-02-22,*The shop was featured on Mind of a Chef with ...,4,3,4,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
2,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
3,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
4,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
5,i5kyjrSfzAC-RJLTYx6pTQ,pnASSACWj87icFTteMkbtA,ACFtxLv8pGrrxMm6EgjreA,5,2014-06-28,"Wow, just the most tasty burguer I've ever had...",0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
6,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
7,HRPm3vEZ_F-33TYVT7Pebw,_4iMDXbXZ1p1ONG297YEAQ,8QWPlVQ6D-OExqXoaD2Z1g,5,2014-09-24,Cycle Pub Las Vegas was a blast! Got a groupon...,1,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,True
8,ymAUG8DZfQcFTBSOiaNN4w,u0LXt3Uea_GidxRW1xcsfg,9_CGhHMz8698M9-PkVf0CQ,4,2012-05-11,Who would have guess that you would be able to...,0,0,2,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False
9,8UIishPUD92hXtScSga_gw,u0LXt3Uea_GidxRW1xcsfg,gkCorLgPyQLsptTHalL61g,4,2015-10-27,Always drove past this coffee house and wonder...,1,0,0,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,False


In [214]:
#Filter all restaurants by is_bougie true
is_bougie = yelp_reviewd[yelp_reviewd["is_bougie"] == True]
text_data_seriesb = is_bougie["text"]
text_data_seriesb.head(10)


0     Super simple place but amazing nonetheless. It...
1     *The shop was featured on Mind of a Chef with ...
2     Small unassuming place that changes their menu...
3     Lester's is located in a beautiful neighborhoo...
5     Wow, just the most tasty burguer I've ever had...
7     Cycle Pub Las Vegas was a blast! Got a groupon...
16    its delicious. its loud. its funny. seems like...
17    Love this place! If I ever have international ...
19    This place is awesome! Definitely authentic!!!...
28    Best burger place I've had in Toronto. Hands d...
Name: text, dtype: object

In [215]:
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
wordsb = (text_data_seriesb
           .str.lower()
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rsltb = pd.DataFrame(Counter(wordsb).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rsltb)

# plot
#rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

              Frequency
Word                   
'                  6016
.                  3062
great              1548
,                  1410
place              1399
food               1074
like               1050
one                1030
get                1020
good                935
love                901
really              855
also                833
go                  825
time                820
-                   816
always              790
!                   785
best                715
back                711
would               707
service             683
nice                588
even                569
(                   538
definitely          534
little              525
us                  517
got                 515
first               514
...                 ...
confusing             2
maze                  2
upstairs,             2
capone                2
cognac                2
symbol                2
structure.            2
modern-age            2
escalators      

In [216]:
#Repeat steps 5-8 with non-matching restaurants
#Filter all restaurants by is_bougie true
is_not_bougie = yelp_reviewd[yelp_reviewd["is_bougie"] == False]
text_data_seriesp = is_not_bougie["text"]
text_data_seriesp.head(10)

4     Love coming here. Yes the place always needs t...
6     Had their chocolate almond croissant and it wa...
8     Who would have guess that you would be able to...
9     Always drove past this coffee house and wonder...
10    Not bad!! Love that there is a gluten-free, ve...
11    I'm not on the Poutini's train. It's fine but ...
12    This is currently my parents new favourite res...
13    Server was a little rude.\n\nOrdered the calam...
14    I thought Tidy's Flowers had a great reputatio...
15    Wanted to check out this place due to all the ...
Name: text, dtype: object

In [217]:
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
wordsp = (text_data_seriesp
           .str.lower()
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rsltp = pd.DataFrame(Counter(wordsp).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rsltp)

# plot
#rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

              Frequency
Word                   
'                 16033
.                  7431
,                  3810
like               3155
food               2910
place              2902
good               2821
one                2490
get                2485
would              2367
really             2203
-                  1984
great              1880
time               1761
also               1724
go                 1714
service            1692
(                  1584
back               1531
little             1463
ordered            1418
even               1324
got                1318
pretty             1298
nice               1279
came               1242
us                 1117
order              1086
try                1079
could              1077
...                 ...
curried               4
curd                  4
finish,               4
open!                 4
october.              4
minutes...            4
indicating            4
parents.              4
appropriate.    

In [218]:
odd_word = list({s for s in wordsb if s not in wordsp})
print(odd_word)


["toss')!", 'dodgebow', 'kyklos,', 'islands.', 'lake!', 'chiropractor,', 'coras', 'permeable,', 'crabmeat\\/cream', 'condesa', 'wings!!', 'closeting', 'testing!', 'necessity).', 'spot.....', 'cringe', 'rare--', '\\/c,', 'collection.', 'exceptions', 'donkeys', 'gleefully', 'like!"', 'grits!', 'simmons!', "belly'", 'patel', 'poses', 'regularly!!', '-morning', 'tuneup', 'paring', 'winery\\/gathering', 'kosher,', 'jessa', 'thirty-', 'chopped-', 'creamer!!', '(grand', 'torrentially', 'beautifully-decorated', 'earlybird', 'hexagonal', 'integrity.', 'decompression', '(major', 'centric', 'weekly.', 'tension.', "tattoo'", 'exceptional!!', 'notebooks.', 'strudel', 'kahwa"', 'delievery', 'moreso', 'vegi', 'dessertsssss.', 'andrew!', 'giada', 'boar.', "!'", 'journee', 'caffee', 'wrote,', 'modique', 'wowza!', 'eclats', '"cheap.', 'laia!', 'softie.', 'rare...', 'melissa?', 'ikura,', 'subcultures', 'recommend...', 'plumbers(', 'matar', 'impeccable!', 'orienting', 'pistachios..wonderful', 'rositas', '

In [219]:
odd_wordres = pd.DataFrame(Counter(odd_word).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(odd_wordres)


                 Frequency
Word                      
toss')!                  1
dodgebow                 1
kyklos,                  1
islands.                 1
lake!                    1
chiropractor,            1
coras                    1
permeable,               1
crabmeat\/cream          1
condesa                  1
wings!!                  1
closeting                1
testing!                 1
necessity).              1
spot.....                1
cringe                   1
rare--                   1
\/c,                     1
collection.              1
exceptions               1
donkeys                  1
gleefully                1
like!"                   1
grits!                   1
simmons!                 1
belly'                   1
patel                    1
poses                    1
regularly!!              1
-morning                 1
...                    ...
chad!                    1
£18.95                   1
thriving!                1
aspects.                 1
3